O arquivo lambda1_ons_aws.inpy tem como objetivos:

1- Realizar download de arquivos da plataforma ONS de forma temporaria;

2- Enviar os arquivos para um S3 buncket.

In [ ]:
import os
import boto3
import requests
import shutil
from datetime import datetime, timedelta

def lambda_handler(event, context):
    # Configura as informações de data, capturando a data de hoje e 5 dias anteriores
    num_days = 5
    dates = [datetime.today() - timedelta(days=x) for x in range(num_days)][::-1]
    date_formats = [date.strftime('%Y_%m_%d') for date in dates]
    date_formats2 = [date.strftime('%d-%m-%Y') for date in dates]

    # Cria o diretório temporário para salvar os arquivos
    tmp_dir = '/tmp/ons-data'
    os.makedirs(tmp_dir, exist_ok=True)


    # Baixa e envia arquivos para o bucket S3
    s3 = boto3.client('s3')
    bucket_name = 's3lambdaons'
    files = []
    url_regioes = {}
    for i in range(len(date_formats)):
        url_regioes[date_formats[i]] = {
            'sul': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/03_DadosDiariosAcumulados_Regiao_{date_formats2[i]}.xlsx',
            'sudeste': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/04_DadosDiariosAcumulados_Regiao_{date_formats2[i]}.xlsx',
            'nordeste': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/05_DadosDiariosAcumulados_Regiao_{date_formats2[i]}.xlsx',
            'norte': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/06_DadosDiariosAcumulados_Regiao_{date_formats2[i]}.xlsx',
            'brasil': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/07_DadosDiariosAcumulados_Regiao_{date_formats2[i]}.xlsx'
        }
        for regiao, url in url_regioes[date_formats[i]].items():
            file_name = f'BEA_D_{regiao.upper()}_{date_formats[i]}.xlsx'
            try:
                response = requests.get(url)
                response.raise_for_status()
                file_path = os.path.join(tmp_dir, file_name)
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                s3.upload_file(file_path, bucket_name, f'BEA/{file_name}')     #define pasta de envio
                files.append(file_path)
            except requests.exceptions.RequestException as e:
                print(f'Erro ao baixar arquivo {file_name}: {e}')
    
    # PROD_D
    url_prod = {}
    for i in range(len(date_formats)):
        url_prod[date_formats[i]] = {
            '_H': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/08_ProducaoHidraulicaUsina_{date_formats2[i]}.xlsx',
            '_T': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/09_ProducaoTermicaUsina_{date_formats2[i]}.xlsx'
        }
        for prod_type, url in url_prod[date_formats[i]].items():
            file_name = f'BDE_D_PROD_{prod_type}_{date_formats[i]}.xlsx'
            try:
                response = requests.get(url)
                response.raise_for_status()
                file_path = os.path.join(tmp_dir, file_name)
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                s3.upload_file(file_path, bucket_name, f'PROD_D/{file_name}')     #define pasta de envio
                files.append(file_path)
            except requests.exceptions.RequestException as e:
                print(f'Erro ao baixar arquivo {file_name}: {e}')
    
    # RESERVATORIOS
    url_reservatorios ={}
    for i in range(len(date_formats)):
        url_reservatorios[date_formats[i]] = {
            'sul': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/23_SituacaoPrincipaisReservatorios_Regiao_{date_formats2[i]}.xlsx',
            'sudeste': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/24_SituacaoPrincipaisReservatorios_Regiao_{date_formats2[i]}.xlsx',
            'nordeste': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/25_SituacaoPrincipaisReservatorios_Regiao_{date_formats2[i]}.xlsx',
            'norte': f'https://sdro.ons.org.br/SDRO/DIARIO/{date_formats[i]}/HTML/26_SituacaoPrincipaisReservatorios_Regiao_{date_formats2[i]}.xlsx'
        }
        for reservatorios, url in url_reservatorios[date_formats[i]].items():
            file_name = f'RES_D_{reservatorios}_{date_formats[i]}.xlsx'
            try:
                response = requests.get(url)
                response.raise_for_status()
                file_path = os.path.join(tmp_dir, file_name)
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                s3.upload_file(file_path, bucket_name, f'RES_D/{file_name}')     #define pasta de envio
                files.append(file_path)
            except requests.exceptions.RequestException as e:
                print(f'Erro ao baixar arquivo {file_name}: {e}')        
    
    
    
    
    # Remove os arquivos do diretório temporário
    for file in files:
        os.remove(file)
    shutil.rmtree(tmp_dir)
    
    # Retorna uma mensagem de sucesso
    return {
        'statusCode': 200,
        'body': 'Os arquivos foram baixados e enviados para o bucket S3 com sucesso.'
    }